In [1]:
import pandas as pd

data = pd.read_csv('file:///home/vishalthadari/Downloads/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [3]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [6]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [24]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/vishalthadari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
Stemmer = SnowballStemmer("english")

In [32]:
def lemmatize_stemming(text):
    return Stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [33]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
doc_sample

'rain helps dampen bushfires'

In [34]:
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)

original document: 


In [35]:
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [36]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [42]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [38]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [39]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [46]:
bow_doc_4310 = bow_corpus[4311]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 167 ("govt") appears 1 time.
Word 245 ("group") appears 1 time.
Word 298 ("vote") appears 1 time.
Word 601 ("local") appears 1 time.
Word 866 ("want") appears 1 time.
Word 3878 ("compulsori") appears 1 time.
Word 3879 ("ratepay") appears 1 time.


In [47]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908644709983),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.5046520328695662)]


In [48]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908644709983),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.5046520328695662)]


In [49]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [50]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.018*"market" + 0.013*"turnbul" + 0.013*"share" + 0.012*"break" + 0.012*"deal" + 0.012*"life" + 0.012*"rise" + 0.011*"worker" + 0.011*"bank" + 0.010*"china"
Topic: 1 
Words: 0.021*"north" + 0.018*"help" + 0.017*"state" + 0.012*"farmer" + 0.012*"leagu" + 0.011*"victoria" + 0.011*"busi" + 0.010*"centr" + 0.009*"game" + 0.009*"john"
Topic: 2 
Words: 0.020*"canberra" + 0.018*"die" + 0.016*"rural" + 0.016*"hospit" + 0.015*"donald" + 0.014*"dead" + 0.013*"work" + 0.012*"melbourn" + 0.012*"train" + 0.011*"crash"
Topic: 3 
Words: 0.033*"trump" + 0.021*"court" + 0.019*"face" + 0.017*"alleg" + 0.016*"health" + 0.014*"say" + 0.012*"servic" + 0.012*"trial" + 0.011*"tell" + 0.010*"peopl"
Topic: 4 
Words: 0.046*"polic" + 0.027*"charg" + 0.021*"death" + 0.020*"murder" + 0.017*"woman" + 0.017*"perth" + 0.014*"brisban" + 0.014*"interview" + 0.014*"jail" + 0.013*"miss"
Topic: 5 
Words: 0.022*"kill" + 0.015*"tasmania" + 0.013*"island" + 0.011*"citi" + 0.010*"attack" + 0.010*"final" + 0.

In [51]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.016*"charg" + 0.014*"murder" + 0.013*"polic" + 0.011*"court" + 0.010*"alleg" + 0.009*"jail" + 0.008*"assault" + 0.008*"arrest" + 0.007*"accus" + 0.007*"woman"
Topic: 1 Word: 0.012*"trump" + 0.009*"govern" + 0.007*"rural" + 0.007*"health" + 0.007*"fund" + 0.006*"council" + 0.005*"budget" + 0.005*"plan" + 0.005*"cut" + 0.005*"say"
Topic: 2 Word: 0.010*"turnbul" + 0.008*"grandstand" + 0.007*"ash" + 0.007*"plead" + 0.006*"wednesday" + 0.005*"kill" + 0.005*"syria" + 0.005*"russia" + 0.005*"suicid" + 0.004*"legal"
Topic: 3 Word: 0.010*"interview" + 0.009*"weather" + 0.007*"australia" + 0.006*"world" + 0.006*"leagu" + 0.006*"monday" + 0.006*"rugbi" + 0.005*"cricket" + 0.005*"mount" + 0.005*"test"
Topic: 4 Word: 0.008*"tasmania" + 0.007*"climat" + 0.005*"toni" + 0.005*"capit" + 0.005*"jam" + 0.004*"onlin" + 0.004*"quarter" + 0.004*"kid" + 0.004*"interview" + 0.004*"paul"
Topic: 5 Word: 0.011*"drum" + 0.007*"friday" + 0.006*"michael" + 0.006*"asylum" + 0.006*"wall" + 0.005*"see

In [52]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [53]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8199950456619263	 
Topic: 0.021*"north" + 0.018*"help" + 0.017*"state" + 0.012*"farmer" + 0.012*"leagu" + 0.011*"victoria" + 0.011*"busi" + 0.010*"centr" + 0.009*"game" + 0.009*"john"

Score: 0.020004184916615486	 
Topic: 0.046*"polic" + 0.027*"charg" + 0.021*"death" + 0.020*"murder" + 0.017*"woman" + 0.017*"perth" + 0.014*"brisban" + 0.014*"interview" + 0.014*"jail" + 0.013*"miss"

Score: 0.020000644028186798	 
Topic: 0.033*"trump" + 0.021*"court" + 0.019*"face" + 0.017*"alleg" + 0.016*"health" + 0.014*"say" + 0.012*"servic" + 0.012*"trial" + 0.011*"tell" + 0.010*"peopl"

Score: 0.02000010386109352	 
Topic: 0.024*"world" + 0.022*"south" + 0.021*"australian" + 0.019*"open" + 0.017*"test" + 0.017*"countri" + 0.016*"australia" + 0.014*"hour" + 0.013*"sydney" + 0.013*"arrest"

Score: 0.019999997690320015	 
Topic: 0.018*"market" + 0.013*"turnbul" + 0.013*"share" + 0.012*"break" + 0.012*"deal" + 0.012*"life" + 0.012*"rise" + 0.011*"worker" + 0.011*"bank" + 0.010*"china"

Score: 0.

In [54]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6390663981437683	 
Topic: 0.010*"interview" + 0.009*"weather" + 0.007*"australia" + 0.006*"world" + 0.006*"leagu" + 0.006*"monday" + 0.006*"rugbi" + 0.005*"cricket" + 0.005*"mount" + 0.005*"test"

Score: 0.20090974867343903	 
Topic: 0.014*"crash" + 0.009*"polic" + 0.008*"miss" + 0.007*"search" + 0.007*"die" + 0.006*"marriag" + 0.006*"road" + 0.006*"woman" + 0.006*"truck" + 0.006*"death"

Score: 0.020006900653243065	 
Topic: 0.012*"trump" + 0.009*"govern" + 0.007*"rural" + 0.007*"health" + 0.007*"fund" + 0.006*"council" + 0.005*"budget" + 0.005*"plan" + 0.005*"cut" + 0.005*"say"

Score: 0.02000351995229721	 
Topic: 0.011*"drum" + 0.007*"friday" + 0.006*"michael" + 0.006*"asylum" + 0.006*"wall" + 0.005*"seeker" + 0.005*"smith" + 0.005*"blog" + 0.005*"anim" + 0.005*"fiji"

Score: 0.02000277303159237	 
Topic: 0.021*"countri" + 0.019*"hour" + 0.010*"podcast" + 0.006*"octob" + 0.006*"novemb" + 0.005*"stori" + 0.005*"peter" + 0.005*"quiz" + 0.005*"andrew" + 0.004*"queensland"

Score

In [55]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3500001132488251	 Topic: 0.018*"market" + 0.013*"turnbul" + 0.013*"share" + 0.012*"break" + 0.012*"deal"
Score: 0.1833333522081375	 Topic: 0.021*"north" + 0.018*"help" + 0.017*"state" + 0.012*"farmer" + 0.012*"leagu"
Score: 0.1833333522081375	 Topic: 0.020*"canberra" + 0.018*"die" + 0.016*"rural" + 0.016*"hospit" + 0.015*"donald"
Score: 0.18333317339420319	 Topic: 0.025*"elect" + 0.021*"plan" + 0.016*"council" + 0.013*"school" + 0.012*"hous"
Score: 0.01666666753590107	 Topic: 0.033*"trump" + 0.021*"court" + 0.019*"face" + 0.017*"alleg" + 0.016*"health"
Score: 0.01666666753590107	 Topic: 0.046*"polic" + 0.027*"charg" + 0.021*"death" + 0.020*"murder" + 0.017*"woman"
Score: 0.01666666753590107	 Topic: 0.022*"kill" + 0.015*"tasmania" + 0.013*"island" + 0.011*"citi" + 0.010*"attack"
Score: 0.01666666753590107	 Topic: 0.024*"world" + 0.022*"south" + 0.021*"australian" + 0.019*"open" + 0.017*"test"
Score: 0.01666666753590107	 Topic: 0.033*"govern" + 0.025*"year" + 0.015*"child" + 0.0